# Importing Libraries & Data

In [7]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
rawData=pd.read_csv('CrowdstormingDataJuly1st.csv')

# Visualizing rawData

In [9]:
rawData.ix[:10,:13]

,playerShort,player,club,leagueCountry,birthday,height,weight,position,games,victories,ties,defeats,goals
0,lucas-wilchez,Lucas Wilchez,Real Zaragoza,Spain,31.08.1983,177.0,72.0,Attacking Midfielder,1,0,0,1,0
1,john-utaka,John Utaka,Montpellier HSC,France,08.01.1982,179.0,82.0,Right Winger,1,0,0,1,0
2,abdon-prats,Abdón Prats,RCD Mallorca,Spain,17.12.1992,181.0,79.0,NaN,1,0,1,0,0
3,pablo-mari,Pablo Marí,RCD Mallorca,Spain,31.08.1993,191.0,87.0,Center Back,1,1,0,0,0
4,ruben-pena,Rubén Peña,Real Valladolid,Spain,18.07.1991,172.0,70.0,Right Midfielder,1,1,0,0,0
5,aaron-hughes,Aaron Hughes,Fulham FC,England,08.11.1979,182.0,71.0,Center Back,1,0,0,1,0
6,aleksandar-kolarov,Aleksandar Kolarov,Manchester City,England,10.11.1985,187.0,80.0,Left Fullback,1,1,0,0,0
7,alexander-tettey,Alexander Tettey,Norwich City,England,04.04.1986,180.0,68.0,Defensive Midfielder,1,0,0,1,0
8,anders-lindegaard,Anders Lindegaard,Manchester United,England,13.04.1984,193.0,80.0,Goalkeeper,1,0,1,0,0
9,andreas-beck,Andreas Beck,1899 Hoffenheim,Germany,13.03.1987,180.0,70.0,Right Fullback,1,1,0,0,0


The definition of all the "column values" are described in DATA.md but let screen them here also: 

In [10]:
rawData.ix[:10,13:]

,yellowCards,yellowReds,redCards,photoID,rater1,rater2,refNum,refCountry,Alpha_3,meanIAT,nIAT,seIAT,meanExp,nExp,seExp
0,0,0,0,95212.jpg,0.25,0.50,1,1,GRC,0.326391,712.0,0.000564,0.396000,750.0,0.002696
1,1,0,0,1663.jpg,0.75,0.75,2,2,ZMB,0.203375,40.0,0.010875,-0.204082,49.0,0.061504
2,1,0,0,NaN,NaN,NaN,3,3,ESP,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002
3,0,0,0,NaN,NaN,NaN,3,3,ESP,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002
4,0,0,0,NaN,NaN,NaN,3,3,ESP,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002
5,0,0,0,3868.jpg,0.25,0.00,4,4,LUX,0.325185,127.0,0.003297,0.538462,130.0,0.013752
6,0,0,0,47704.jpg,0.00,0.25,4,4,LUX,0.325185,127.0,0.003297,0.538462,130.0,0.013752
7,0,0,0,22356.jpg,1.00,1.00,4,4,LUX,0.325185,127.0,0.003297,0.538462,130.0,0.013752
8,0,0,0,16528.jpg,0.25,0.25,4,4,LUX,0.325185,127.0,0.003297,0.538462,130.0,0.013752
9,0,0,0,36499.jpg,0.00,0.00,4,4,LUX,0.325185,127.0,0.003297,0.538462,130.0,0.013752


# Tests

In [11]:
list(rawData.columns.values)

['playerShort',
 'player',
 'club',
 'leagueCountry',
 'birthday',
 'height',
 'weight',
 'position',
 'games',
 'victories',
 'ties',
 'defeats',
 'goals',
 'yellowCards',
 'yellowReds',
 'redCards',
 'photoID',
 'rater1',
 'rater2',
 'refNum',
 'refCountry',
 'Alpha_3',
 'meanIAT',
 'nIAT',
 'seIAT',
 'meanExp',
 'nExp',
 'seExp']

In [12]:
R=rawData[rawData['player'] == 'Philipp Lahm']
R.head()

,playerShort,player,club,leagueCountry,birthday,height,weight,position,games,victories,...,rater2,refNum,refCountry,Alpha_3,meanIAT,nIAT,seIAT,meanExp,nExp,seExp
1806,philipp-lahm,Philipp Lahm,Bayern München,Germany,11.11.1983,170.0,66.0,Left Fullback,2,2,...,0.25,72,28,IRL,0.355498,4078.0,0.000098,0.517225,4238.0,0.000405
2935,philipp-lahm,Philipp Lahm,Bayern München,Germany,11.11.1983,170.0,66.0,Left Fullback,4,1,...,0.25,88,3,ESP,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002
3748,philipp-lahm,Philipp Lahm,Bayern München,Germany,11.11.1983,170.0,66.0,Left Fullback,2,1,...,0.25,100,50,MKD,0.327139,170.0,0.002362,0.959538,173.0,0.014794
4165,philipp-lahm,Philipp Lahm,Bayern München,Germany,11.11.1983,170.0,66.0,Left Fullback,1,1,...,0.25,113,52,RUS,0.398174,526.0,0.000809,1.212727,550.0,0.004521
5276,philipp-lahm,Philipp Lahm,Bayern München,Germany,11.11.1983,170.0,66.0,Left Fullback,1,1,...,0.25,131,27,ROU,0.385082,537.0,0.000756,0.859431,562.0,0.003620


In [13]:
R=R[R['games']>=40]
R

,playerShort,player,club,leagueCountry,birthday,height,weight,position,games,victories,...,rater2,refNum,refCountry,Alpha_3,meanIAT,nIAT,seIAT,meanExp,nExp,seExp
48893,philipp-lahm,Philipp Lahm,Bayern München,Germany,11.11.1983,170.0,66.0,Left Fullback,47,29,...,0.25,933,8,DEU,0.336628,7749.0,0.000055,0.335967,7974.0,0.000225
136419,philipp-lahm,Philipp Lahm,Bayern München,Germany,11.11.1983,170.0,66.0,Left Fullback,40,24,...,0.25,2902,8,DEU,0.336628,7749.0,0.000055,0.335967,7974.0,0.000225


# Analyzing Data

After looking at the data, we realized that some referees had less than 22 Dayds which is technically impossible since at every game each referee receive at least 22 Dyads. So we decied to remove the referees with less than 22 Dyads

In [128]:
rawData.sort_values('refNum');
allRefs = len(rawData.refNum.value_counts())
allDyads = np.sum(rawData.games)

specRef=rawData
for i in range(allRefs):
    specRef1=specRef[specRef['refNum']==i] #we put dataframe of each referee into specRef1 as a temporary variable
    if(np.sum(specRef1['games'])<22):
        specRef=specRef.drop(specRef1.index)

,playerShort,player,club,leagueCountry,birthday,height,weight,position,games,victories,...,rater2,refNum,refCountry,Alpha_3,meanIAT,nIAT,seIAT,meanExp,nExp,seExp
5,aaron-hughes,Aaron Hughes,Fulham FC,England,08.11.1979,182.0,71.0,Center Back,1,0,...,0.00,4,4,LUX,0.325185,127.0,0.003297,0.538462,130.0,0.013752
6,aleksandar-kolarov,Aleksandar Kolarov,Manchester City,England,10.11.1985,187.0,80.0,Left Fullback,1,1,...,0.25,4,4,LUX,0.325185,127.0,0.003297,0.538462,130.0,0.013752
7,alexander-tettey,Alexander Tettey,Norwich City,England,04.04.1986,180.0,68.0,Defensive Midfielder,1,0,...,1.00,4,4,LUX,0.325185,127.0,0.003297,0.538462,130.0,0.013752
8,anders-lindegaard,Anders Lindegaard,Manchester United,England,13.04.1984,193.0,80.0,Goalkeeper,1,0,...,0.25,4,4,LUX,0.325185,127.0,0.003297,0.538462,130.0,0.013752
9,andreas-beck,Andreas Beck,1899 Hoffenheim,Germany,13.03.1987,180.0,70.0,Right Fullback,1,1,...,0.00,4,4,LUX,0.325185,127.0,0.003297,0.538462,130.0,0.013752


In [135]:
specRef.head()  #specRef is now containing the new DataFrame without the referees with less than 22 Dyads, corresponding to one game as a referee.
specRef=specRef.reset_index(drop=True)  #To reset the index of the Dataframe

Now let's study the two parameters 'rater1' and 'rater2' to see their robustness to define if a player is "black" or "white". And to do so, lets visualize the two raters first.

In [137]:
specRef.ix[:100,17:19]
specRef.hist(data, column=None, by=None, grid=True, xlabelsize=None, ylabelsize=None, ax=None, bins=10)

,rater1,rater2
0,0.25,0.00
1,0.00,0.25
2,1.00,1.00
3,0.25,0.25
4,0.00,0.00
5,0.00,0.00
6,0.50,0.50
7,0.00,0.00
8,0.00,0.00
9,0.00,0.25
